In [81]:
%pip install pandas
%pip install geopandas
%pip install dill
%pip install bokeh
%pip install panel
%pip install jupyter_bokeh

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [82]:
import pandas as pd
import geopandas as gpd

import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import GeoJSONDataSource, HoverTool, ColorBar, LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.transform import factor_cmap
from bokeh.palettes import Blues9, RdYlGn11, Viridis256


In [83]:
from data_loader import get_geo_data, get_citibike_data, get_live_data

car_data, nbh_data = get_geo_data()
items_df, station_df = get_citibike_data()

merged_df = pd.merge(items_df, station_df, on="station_id")

car_df = pd.DataFrame(car_data)
print(merged_df)


      num_docks_available  is_returning  is_renting  is_installed  \
0                       0             0           0             0   
1                       0             0           0             0   
2                       0             0           0             0   
3                       0             0           0             0   
4                       0             0           0             0   
...                   ...           ...         ...           ...   
2316                   27             1           1             1   
2317                    2             1           1             1   
2318                    1             1           1             1   
2319                   13             1           1             1   
2320                   20             1           1             1   

                legacy_id                            station_id  \
0                    4774  41495491-5d89-4e14-aab9-c3db04aad399   
1     1905837242740508940            

### Merging Data:

In [84]:
import numpy as np


bike_gdf = gpd.GeoDataFrame(
    merged_df,
    geometry=gpd.points_from_xy(merged_df.lon, merged_df.lat))
bike_gdf = bike_gdf.set_crs("EPSG:4326", allow_override=True)
bike_gdf = bike_gdf.to_crs(epsg=3857)
merged_gdf = gpd.sjoin(bike_gdf, car_data[['geometry', 'name']], predicate='within')
merged_gdf = merged_gdf.rename(columns={'name_right': 'zip_code'})


station_counts = merged_gdf.groupby('zip_code').size().reset_index(name='station_count')
#print(station_counts)
merged_data = car_data.merge(station_counts, left_on='name', right_on='zip_code', how='left')
merged_data = merged_data.rename(columns={'station_count_y': 'station_count'})

merged_data = merged_data.drop(columns=['zip_code'])
merged_data = merged_data.drop(columns=[col for col in car_data.columns if col.startswith('B25')])
merged_data['station_count'] = merged_data['station_count'].fillna(0)
print(merged_data)

merged_data['car_class'] = pd.qcut(merged_data['pct_vehicle'], 3, labels=[0,1,2], duplicates='drop')
merged_data['bike_class'] = pd.cut(merged_data['station_count'], bins=[-1, 0, 10, np.inf], labels=[0, 1, 2])


merged_data['bivariate_class'] = merged_data['car_class'].astype(int) * 3 + merged_data['bike_class'].astype(int)
print(merged_data)

            geoid   name  households  \
0    86000US10001  10001     15097.0   
1    86000US10002  10002     35771.0   
2    86000US10003  10003     25080.0   
3    86000US10004  10004      1775.0   
4    86000US10005  10005      5156.0   
..            ...    ...         ...   
191  86000US11691  11691     22382.0   
192  86000US11692  11692      7981.0   
193  86000US11693  11693      5405.0   
194  86000US11694  11694      8046.0   
195  86000US11697  11697      1668.0   

                                              geometry  no_vehicle  \
0    MULTIPOLYGON (((-8238766.757 4975921.508, -823...     12699.0   
1    MULTIPOLYGON (((-8237364.465 4970257.948, -823...     29144.0   
2    MULTIPOLYGON (((-8237571.742 4972792.277, -823...     20239.0   
3    MULTIPOLYGON (((-8239611.227 4968758.136, -823...      1427.0   
4    MULTIPOLYGON (((-8239034.703 4969185.603, -823...      4488.0   
..                                                 ...         ...   
191  MULTIPOLYGON (((-8214183

In [ ]:
from bokeh.models import Label
from bokeh.models import TapTool, Div
import panel as pn
bivariate_2 = {
        0: '#d3d3d3', 1: '#d8a386', 2: "#dd6a29", 3: "#7ebbd2", 4: "#819185", 5: "#845e29", 6: "#169dd0", 7: "#167984", 8: "#174f28"
    }

merged_data['original_color'] = merged_data['bivariate_class'].map(bivariate_2)
def create_census_map():
    
    # bivariate_colors = {
    # 0: '#e8e8e8', 1: '#ace4e4', 2: '#5ac8c8',
    # 3: '#dfb0d6', 4: '#a5add3', 5: '#5698b9',
    # 6: '#be64ac', 7: '#8c62aa', 8: '#3b4994'
    # }
    
    
    merged_data['color'] = merged_data['bivariate_class'].map(bivariate_2)
    merged_data['line_color'] = '#ffffff'
    merged_data['opacity'] = '0.7'
    car_data_merc = merged_data.to_crs(epsg=3857)
    bounds = car_data_merc.total_bounds
    p = figure(
        title="Transit Access (Bike Vs Cars)",
        x_range=(bounds[0], bounds[2]),
        y_range=(bounds[1], bounds[3]),  
        tools='pan,zoom_in,zoom_out,reset,save,hover',
        width=1200,
        height=800,
        toolbar_location='above'
    )
    
    p.add_tile("CARTODBPOSITRON")
    
    global geosource, car_data_merc_global
    car_data_merc_global = car_data_merc
    geosource = GeoJSONDataSource(geojson=car_data_merc.to_json())
    
    patches = p.patches(
        'xs', 'ys',
        source=geosource,
        fill_color='color',
        fill_alpha='opacity', 
        line_color='line_color',
        line_width=0.5,
        hover_fill_alpha=0.7,
        hover_line_color='color'
    )

    hover = p.select_one(HoverTool)
    hover.tooltips = [
        ('ZIP Code', '@name'),
        ('Households', '@households{0,0}'),
        ('Vehicle Ownership', '@pct_vehicle{0.1f}%'),
        ('Citibike Station Count', '@station_count'),
    ]

    map_width = bounds[2] - bounds[0]
    map_height = bounds[3] - bounds[1]

    
    img_source = ColumnDataSource(data=dict(
    url=['https://i.imgur.com/wOLvJZR.png'],
    x=[bounds[2] - map_width * 0.02],
    y=[bounds[3] - map_width * 0.3],
    w=[map_width * 0.3],
    h=[map_width * 0.3]
    ))

    p.image_url(url='url', x='x', y='y', w='w', h='h', 
    anchor='bottom_left', source=img_source)
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.xgrid.visible = False
    p.ygrid.visible = False
    p.title.text_font_size = '16pt'
    p.title.align = 'center'

    
    
    
    return p

In [86]:
import panel as pn
pn.extension()

filter_radio = pn.widgets.RadioButtonGroup(
    name='Filter Type',
    options=['All ZIPs', 'High Car Ownership', 'Median Car Ownership', 'Low Car Ownership'],
    button_type='primary'
)
filter_radio_bikes = pn.widgets.RadioButtonGroup(
    name='Filter Type',
    options=['All ZIPs', 'High # of Stations', 'Median # of Stations', 'Low # of Stations'],
)
def update_map(event=None):
    print(f"Update called! Car filter: {filter_radio.value}, Bike filter: {filter_radio_bikes.value}")
    filter_type = filter_radio.value
    bike_filter_type = filter_radio_bikes.value
    filtered = car_data_merc_global.copy()
    if filter_type == 'High Car Ownership':
        car_filter = filtered['pct_vehicle'] > 60
    elif filter_type == 'Median Car Ownership':
        car_filter = (filtered['pct_vehicle'] >= 30) & (filtered['pct_vehicle'] <= 60)
    elif filter_type == 'Low Car Ownership':
        car_filter = filtered['pct_vehicle'] < 30
    else:
        car_filter = pd.Series([True] * len(filtered), index=filtered.index)

    
    if bike_filter_type == 'High # of Stations':
        bike_filter = filtered['station_count'] > 10
    elif bike_filter_type == 'Median # of Stations':
        bike_filter = (filtered['station_count'] < 10) & (filtered['station_count'] > 0)
    elif bike_filter_type == 'Low # of Stations':
        bike_filter = filtered['station_count'] == 0
    else:
        bike_filter = pd.Series([True] * len(filtered), index=filtered.index)
    combined_filter = car_filter & bike_filter
    filtered['color'] = filtered['original_color']
    filtered.loc[~combined_filter, 'color'] = '#eeeeee'
   
    if filter_type != 'All ZIPs' or bike_filter_type != 'All ZIPs':
        filtered.loc[~combined_filter, 'color'] = '#eeeeee'
        filtered.loc[~combined_filter, 'opacity'] = '0.2'

    geosource.geojson = filtered.to_json()

filter_radio.param.watch(update_map, 'value')
filter_radio_bikes.param.watch(update_map, 'value')

Watcher(inst=RadioButtonGroup(name='Filter Type', options=['All ZIPs', 'High # of St...], value='All ZIPs'), cls=<class 'panel.widgets.select.RadioButtonGroup'>, fn=<function update_map at 0x12438bed0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [103]:
div = Div(text="""<h3 style="position:absolute; top:0; left: 20; width: 200px; height: 100px; background-color: #666666; z-index:1000">Click on a zip code to see more details.</h3> """,
    width=200, height=100)

dashboard = pn.Column(
    pn.Row(filter_radio, filter_radio_bikes),
    create_census_map(),
    div
)
dashboard.servable()



BokehModel(combine_events=True, render_bundle={'docs_json': {'9cb8ceae-a447-4794-9c80-4fac5a30dd5b': {'version…

### CitiBike Station Status
I used plotly for this because it is definitely simpler and this was going to be a simpler map. It also has better support for scatter plots. I tried adding the scatter plot to the Bokeh/Panel app and all the dots appeared at once with no support for clustering.

In [91]:
def get_popularity(dataframe):
    dataframe['popularity'] = np.where(
        dataframe['capacity'] > 0,
        dataframe['num_bikes_available'] / dataframe['capacity'],
        0
    )
    dataframe['disabled'] = np.where(
        dataframe['capacity'] > 0,
        (dataframe['num_bikes_disabled'] / dataframe['capacity']) * 100,
        0
    )
    return dataframe

In [100]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

merged_df = get_popularity(merged_df)
print(merged_df)


app = Dash()

hover_text = [
    f"<b>{merged_df['name']}</b><br>" +
    f"Bikes Available: {merged_df['num_bikes_available']}<br>" +
    f"Disabled: {merged_df['num_bikes_disabled']}" +
    f"Capacity: {merged_df['capacity']}<br>" 
    
]


fig = go.Figure(data=go.Scattermapbox(
        lon = merged_df['lon'],
        lat = merged_df['lat'],
        text = merged_df['name'],
        hovertemplate = '<b>%{text}</b><extra></extra>',
        mode = 'markers',
        marker = dict(
            size = (1 - merged_df['popularity']) * 25,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            colorscale = 'Plasma',
            cmin = 0,
            color = merged_df['disabled'],
            cmax = merged_df['disabled'].max(),
            colorbar=dict(
                title=dict(
                    text="Percent of Bikes Disabled",
                    side="right"
                )
            )
        )))
fig.update_layout(
    mapbox = dict(
        style = 'carto-positron',
        center = dict(
            lat = merged_df['lat'].mean(),
            lon = merged_df['lon'].mean()
        ),
        zoom = 12
    ),
    height = 700,
    margin = dict(l=0, r=0, t=40, b=0)
)
    
app.layout = [
html.H1(children='Bike Station Issues Map', style={'textAlign':'center'}),
html.Div([
    html.P('Circle size indicates popularity (more available bikes = smaller size).', 
            style={'textAlign': 'center', 'color': '#666', 'fontSize': '14px'})
]),
dcc.Graph(figure=fig)
]
if __name__ == '__main__':
    app.run(debug=True)

      num_docks_available  is_returning  is_renting  is_installed  \
0                       0             0           0             0   
1                       0             0           0             0   
2                       0             0           0             0   
3                       0             0           0             0   
4                       0             0           0             0   
...                   ...           ...         ...           ...   
2316                   27             1           1             1   
2317                    2             1           1             1   
2318                    1             1           1             1   
2319                   13             1           1             1   
2320                   20             1           1             1   

                legacy_id                            station_id  \
0                    4774  41495491-5d89-4e14-aab9-c3db04aad399   
1     1905837242740508940            

/var/folders/_0/kjc2q5c57dx3p0n2wvtmr0dm0000gn/T/ipykernel_86224/945493224.py:21: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



“Bioinformatics and Other Bits - Choropleth Maps with Geopandas, Bokeh and Panel.” Github.io, 15 Nov. 2019, dmnfarrell.github.io/bioinformatics/bokeh-maps. Accessed 14 Dec. 2025.Contributors, Bokeh. 

“Bokeh Documentation.” Docs.bokeh.org, docs.bokeh.org/en/latest/.